#Zadanie 4 (7 pkt)
Celem zadania jest zaimplementowanie algorytmu drzewa decyzyjnego ID3 dla zadania klasyfikacji. Trening i test należy przeprowadzić dla zbioru Iris. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania drzewa dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Implementacja funkcji entropii - **0.5 pkt**
* Implementacja funkcji entropii zbioru - **0.5 pkt**
* Implementacja funkcji information gain - **0.5 pkt**
* Zbudowanie poprawnie działającego drzewa klasyfikacyjnego i przetestowanie go na wspomnianym wcześniej zbiorze testowym. Jeśli w liściu występuje kilka różnych klas, decyzją jest klasa większościowa. Policzenie accuracy i wypisanie parami klasy rzeczywistej i predykcji. - **4 pkt**
* Przeprowadzenie eksperymentów dla różnych głębokości drzew i podziałów zbioru treningowego i testowego (zmiana wartości argumentu test_size oraz usunięcie random_state). W tym przypadku dla każdego eksperymentu należy wykonać kilka uruchomień programu i wypisać dla każdego uruchomienia accuracy. - **1.5 pkt**

In [133]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np
from statistics import median

iris = load_iris()
x = iris.data
y = iris.target

In [206]:
def entropy_func(class_count, num_samples):
    p = class_count / num_samples
    return -p * math.log(p)

class Group:
    def __init__(self, group_classes):
        self.group_classes = list(group_classes)

    def __len__(self):
        return len(self.group_classes)

    def group_entropy(self):
        value = 0
        lenght  = len(self)
        counter = Counter(self.group_classes)
        for valu in counter.values():
            value += entropy_func(valu, lenght)
        return value


class Node:
    def __init__(self, split_feature, split_val, child_node_a=None, child_node_b=None, val=None):
        self.split_feature = split_feature
        self.split_val = split_val
        self.child_node_a = child_node_a
        self.child_node_b = child_node_b
        self.val = val

    def predict(self, data):
        if self.child_node_a and self.child_node_b:
            return self.child_node_a.predict(data) if data[self.split_feature] < self.split_val else self.child_node_b.predict(data)
        else:
            return self.val

class DecisionTreeClassifier:
    def __init__(self):
        self.tree = None

    @staticmethod
    def get_split_entropy(group_a, group_b):
        groupA = Group(group_a)
        groupB = Group(group_b)
        #  return groupA.group_entropy(), groupB.group_entropy()
        return (groupA.group_entropy() * len(groupA) + groupB.group_entropy() * len(groupB)) / (len(groupA) + len(groupB))

    def get_information_gain(self, parent_group, child_group_a, child_group_b):
        parent_value = Group(parent_group).group_entropy()
        # groupA_value, groupB_value = self.get_split_entropy(child_group_a, child_group_b)
        # return parent_value - (len(child_group_a)/len(parent_group) * groupA_value + len(child_group_b)/len(parent_group) * groupB_value)
        return parent_value - self.get_split_entropy(child_group_a, child_group_b)

    def get_best_feature_split(self, feature_value, data, classes):
        temp_data = [i[feature_value] for i in data]
        split_value = 0
        gain = 0
        for ev_value in set(temp_data):
            clas_a = np.array([])
            clas_b = np.array([])
            for value, clas in zip(temp_data, classes):
                if value < ev_value:
                    clas_a = np.append(clas_a, clas)
                else:
                    clas_b = np.append(clas_b, clas)
            new_gain = self.get_information_gain(classes, clas_a, clas_b)
            if new_gain > gain:
                gain = new_gain
                split_value = ev_value
        return split_value, gain

    def get_best_split(self, data, classes):
        best_gain = 0
        best_split_value = 0
        best_feature_split = 0
        for split_feature in [0, 1, 2, 3]:
            split_value, gain = self.get_best_feature_split(split_feature, data, classes)
            if gain > best_gain:
                best_gain = gain
                best_split_value = split_value
                best_feature_split = split_feature
        return best_split_value, best_feature_split

    def build_tree(self, data, classes, depth):
        if len(set(classes)) == 1:
            return Node(None, None, val=classes[0])
        if depth == 0:
            counter = Counter(classes)
            val = list(counter.keys())[0]
            return Node(None, None, val=val)
        else:
            ###########################################################
             #                    mediana                             #
            ###########################################################
            # gain = 0
            # feature_split = 0
            # split_val = 0
            # for feature in [0, 1, 2, 3]:
            #     parent_data = [i[feature] for i in data]
            #     temp_split_val = median(parent_data)
            #     group_a = []
            #     group_b = []
            #     for val, clas in zip(parent_data, classes):
            #         if val < temp_split_val:
            #             group_a.append(clas)
            #         else:
            #             group_b.append(clas)
            #     new_gain = self.get_information_gain(classes, group_a, group_b)
            #     if new_gain > gain:
            #         gain = new_gain
            #         feature_split = feature
            #         split_val = temp_split_val
            split_val, feature_split = self.get_best_split(data, classes)
            data_a = []
            data_b = []
            classes_a = []
            classes_b = []
            for line, clas in zip(data, classes):
                if line[feature_split] < split_val:
                    data_a.append(line)
                    classes_a.append(clas)
                else:
                    data_b.append(line)
                    classes_b.append(clas)
            return Node(feature_split, split_val, self.build_tree(np.array(data_a), np.array(classes_a), depth-1), self.build_tree(np.array(data_b), np.array(classes_b), depth-1))

    def predict(self, data):
        return self.tree.predict(data)

In [204]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)
dc = DecisionTreeClassifier()
dc.tree = dc.build_tree(x_train, y_train, 3)
count = 0
for sample, gt in zip(x_test, y_test):
    prediction = dc.predict(sample)
    count += prediction==gt
print(count/len(x_test))

0.9333333333333333


In [208]:
i = 5
while i>0:  
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
    dc = DecisionTreeClassifier()
    dc.tree = dc.build_tree(x_train, y_train, 3)
    count = 0
    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        count += prediction==gt
    print(count/len(x_test))
    i -= 1

0.8666666666666667
1.0
0.9333333333333333
0.9333333333333333
0.9333333333333333


In [155]:
i = 5
while i>0:  
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
    dc = DecisionTreeClassifier()
    dc.tree = dc.build_tree(x_train, y_train, 4)
    count = 0
    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        count += prediction==gt
    print(count/len(x_test))
    i -= 1

0.9333333333333333
0.8
0.9333333333333333
1.0
1.0


In [154]:
i = 5
while i>0:  
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
    dc = DecisionTreeClassifier()
    dc.tree = dc.build_tree(x_train, y_train, 5)
    count = 0
    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        count += prediction==gt
    print(count/len(x_test))
    i -= 1

0.9333333333333333
0.9333333333333333
1.0
0.9333333333333333
0.9333333333333333


In [198]:
i = 5
while i>0:  
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
    dc = DecisionTreeClassifier()
    dc.tree = dc.build_tree(x_train, y_train, 6)
    count = 0
    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        count += prediction==gt
    print(count/len(x_test))
    i -= 1

0.8666666666666667
0.9333333333333333
1.0
1.0
1.0


In [147]:
i = 5
while i>0:  
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    dc = DecisionTreeClassifier()
    dc.tree = dc.build_tree(x_train, y_train, 3)
    count = 0
    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        count += prediction==gt
    print(count/len(x_test))
    i -= 1

0.9
0.9
0.9666666666666667
0.9666666666666667
0.8666666666666667


In [158]:
i = 5
while i>0:  
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    dc = DecisionTreeClassifier()
    dc.tree = dc.build_tree(x_train, y_train, 4)
    count = 0
    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        count += prediction==gt
    print(count/len(x_test))
    i -= 1

0.9333333333333333
0.9
0.9
0.9
0.9666666666666667


In [144]:
i = 5
while i>0:  
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    dc = DecisionTreeClassifier()
    dc.tree = dc.build_tree(x_train, y_train, 5)
    count = 0
    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        count += prediction==gt
    print(count/len(x_test))
    i -= 1

1.0
0.9
1.0
0.8333333333333334
0.9


In [175]:
i = 5
while i>0:  
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    dc = DecisionTreeClassifier()
    dc.tree = dc.build_tree(x_train, y_train, 6)
    count = 0
    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        count += prediction==gt
    print(count/len(x_test))
    i -= 1

0.9666666666666667
0.9
0.9
0.9333333333333333
0.9333333333333333
